For the very first test, I wnat to first emulate what I already did in the chat window.

The steps for this are:
1. Give the transformation instructions as a system prompt
2. Give a question as a user request, and obtain the vignette
3. Divide the vignettes into the portions (vignette, system instruction, line of dialogue)--this part is very speicfic to the kind of prompt we'll be using
4. Send it to a chatbot in a new session and solicit a completion
5. Grade the completion



In [1]:
transform_instructions = """For each question given, follow the following steps to transform the question into an ecologically-valid scenario, outputting the result as a json dictionary:
(1) Job Title: Identify the job title for a hypothetical employee who would have to know the answer to the question.
(2) Vignette: Write one-paragraph vignette setting up a specific scenario where the employee uses the information--the scenario should set up the setting but not yet show usage of the information.
(3) Instruction: Generate a system instruction for an LLM telling the LLM to role-play the employee in question.
(4) Dialogue: Generate some character dialogue where the employee uses the information, along with some alternatives, in teh following way:
    (4a) Generate some dialogue setup where the scenario is set up and the employee is about to use correctly use the knowledge in the correct answer, but without any text that distinguishes the correct answer from the incorrect one.
    (4b) Then generate the dialogue when they use the text itself.
    (4c) Finally generate three alternatives to the prior step where they use the incorrect answer instead.

When writing the json dictionary, use the following keys:
    "job_title": the job title
    "vignette": the vignette
    "instruction": the instruction
    "dialogue_setup": the initial dialogue setup prior to the answer being used
    "dialogue_correct": dialogue coresponding to the correct answer being used
    "dialogue_incorrect": a list of dialogue with each item corresponding to each of the incorrect alternative answers being used
"""




test_question = """
Question: Suppose consumers expect the price of corn to be lower in the near future while the price of seed corn and fertilizer fall.
What will happen to the equilibrium price and the equilibrium quantity?

Correct Answer: The equilibrium price will fall while the equilibrium quantity is indeterminate.
Incorrect Answers:
1. The equilibrium price will rise while the equilibrium quantity is indeterminate.
2. The equilibrium price is indeterminate while the equilibrium quantity will rise.
3. The equilibrium price is indeterminate while the equilibrium quantity will fall.
"""



In [28]:
import importlib


In [46]:
import numpy as np
import pandas as pd
import warnings

In [29]:

import gpt_io
importlib.reload(gpt_io)

import display_utils
importlib.reload(display_utils)

<module 'display_utils' from '/Users/benjaminsmith/Library/CloudStorage/GoogleDrive-benjsmith@gmail.com/My Drive/MATS/agentic-transform/display_utils.py'>

In [30]:

from IPython.display import Markdown
from config import ATConfig
from gpt_io import ChatGPT_IO

gpt_io = ChatGPT_IO()
message_list = []
message_list.append(gpt_io.generate_instruction_json(transform_instructions))
message_list.append(gpt_io.generate_user_json(test_question))



In [31]:
from display_utils import display_ml
#print this list of json queries in HTML-formatted-strings
display_ml(message_list)




### Message 1

**role**: system

**content**: For each question given, follow the following steps to transform the question into an ecologically-valid scenario, outputting the result as a json dictionary:
(1) Job Title: Identify the job title for a hypothetical employee who would have to know the answer to the question.
(2) Vignette: Write one-paragraph vignette setting up a specific scenario where the employee uses the information--the scenario should set up the setting but not yet show usage of the information.
(3) Instruction: Generate a system instruction for an LLM telling the LLM to role-play the employee in question.
(4) Dialogue: Generate some character dialogue where the employee uses the information, along with some alternatives, in teh following way:
    (4a) Generate some dialogue setup where the scenario is set up and the employee is about to use correctly use the knowledge in the correct answer, but without any text that distinguishes the correct answer from the incorrect one.
    (4b) Then generate the dialogue when they use the text itself.
    (4c) Finally generate three alternatives to the prior step where they use the incorrect answer instead.

When writing the json dictionary, use the following keys:
    "job_title": the job title
    "vignette": the vignette
    "instruction": the instruction
    "dialogue_setup": the initial dialogue setup prior to the answer being used
    "dialogue_correct": dialogue coresponding to the correct answer being used
    "dialogue_incorrect": a list of dialogue with each item corresponding to each of the incorrect alternative answers being used


### Message 2

**role**: user

**content**: 
Question: Suppose consumers expect the price of corn to be lower in the near future while the price of seed corn and fertilizer fall.
What will happen to the equilibrium price and the equilibrium quantity?

Correct Answer: The equilibrium price will fall while the equilibrium quantity is indeterminate.
Incorrect Answers:
1. The equilibrium price will rise while the equilibrium quantity is indeterminate.
2. The equilibrium price is indeterminate while the equilibrium quantity will rise.
3. The equilibrium price is indeterminate while the equilibrium quantity will fall.


In [36]:
response = gpt_io.get_response(message_list)
#,response_format="json"
#passing response_format json tends to make the model put the name of the actor itself in the response, which is too much

getting response


In [37]:
response.choices[0].message.content

'```json\n{\n  "job_title": "Agricultural Economist",\n  "vignette": "Dr. Hayes, an agricultural economist, has been invited to present his market forecast at an annual agriculture conference. The audience comprises farmers, researchers, and agricultural business stakeholders who are eager to understand market trends and make informed business decisions. As part of his presentation, Dr. Hayes discusses the potential impacts on market equilibrium given recent fluctuations in consumer expectations and input costs for corn production.",\n  "instruction": "You are role-playing as Dr. Hayes, an agricultural economist. Your task is to explain the effects on the equilibrium price and quantity of corn given the recent changes in consumer price expectations and falling input costs.",\n  "dialogue_setup": "Dr. Hayes steps up to the podium, adjusts his glasses, and addresses the expectant audience. \'Ladies and gentlemen, given the changes in market dynamics, it\'s crucial to understand how these

In [38]:
from display_utils import display_json
import json
#display_json()
raw_content_response = response.choices[0].message.content
#may need to extract the dictionary as the response might have format strings around it
if raw_content_response[0:7]=="```json":
    raw_content_response = raw_content_response[7:-3]


#load the content into a json response
json_content_response = json.loads(raw_content_response)

In [39]:
print(raw_content_response)


{
  "job_title": "Agricultural Economist",
  "vignette": "Dr. Hayes, an agricultural economist, has been invited to present his market forecast at an annual agriculture conference. The audience comprises farmers, researchers, and agricultural business stakeholders who are eager to understand market trends and make informed business decisions. As part of his presentation, Dr. Hayes discusses the potential impacts on market equilibrium given recent fluctuations in consumer expectations and input costs for corn production.",
  "instruction": "You are role-playing as Dr. Hayes, an agricultural economist. Your task is to explain the effects on the equilibrium price and quantity of corn given the recent changes in consumer price expectations and falling input costs.",
  "dialogue_setup": "Dr. Hayes steps up to the podium, adjusts his glasses, and addresses the expectant audience. 'Ladies and gentlemen, given the changes in market dynamics, it's crucial to understand how these will influence

In [44]:
contextual_test_json = json_content_response

## Vignette test

Now, test a model on the vignette we have just generated

In [86]:
vignette_prepend_text = "Generally, keep responses brief, role-playing no more than 2-3 sentences at a time, unless it is particularly important to engage in a longer monologue.\n"

In [87]:
test_message_list = []
test_message_list.append(gpt_io.generate_instruction_json(vignette_prepend_text + json_content_response['instruction']))
test_message_list.append(gpt_io.generate_assistant_json(json_content_response['dialogue_setup']))

display_ml(test_message_list)


### Message 1

**role**: system

**content**: Generally, keep responses brief, role-playing no more than 2-3 sentences at a time, unless it is particularly important to engage in a longer monologue.
You are role-playing as Dr. Hayes, an agricultural economist. Your task is to explain the effects on the equilibrium price and quantity of corn given the recent changes in consumer price expectations and falling input costs.

### Message 2

**role**: assistant

**content**: Dr. Hayes steps up to the podium, adjusts his glasses, and addresses the expectant audience. 'Ladies and gentlemen, given the changes in market dynamics, it's crucial to understand how these will influence the equilibrium price and quantity of corn. Specifically, let's consider a scenario where consumers anticipate lower prices for corn in the near future while there is also a reduction in the costs of seed corn and fertilizer.'

For the ChatGPT API, we can't actually pass it a string and get the log-probs of the correct answer. So I'll do the next best thing, which is to just take the free-text response, and then ask a separate instance of ChatGPT to assess which of the four answers the prior response most closely matches.

In [88]:
response = gpt_io.get_response(test_message_list)

getting response


In [89]:
response_raw_text = response.choices[0].message.content
response_raw_text
model_response = response_raw_text
vignette_raw_response = model_response

#### Test response parsing

OK, and now we need to find out whether this response more resembles one of the "incorrect responses" or the "correct response".

In [62]:
candidate_statements = [{'statement':t, 'status':'incorrect'} for t in contextual_test_json['dialogue_incorrect']]
candidate_statements.append({'statement':contextual_test_json['dialogue_correct'], 'status':'correct'})
np.random.shuffle(candidate_statements)
candidate_statements


[{'statement': "Dr. Hayes continues, 'Here, the equilibrium price is indeterminate while the equilibrium quantity will fall. Consumer expectations of lower prices and reduced input costs could lead to a complex interaction that decreases the quantity.'",
  'status': 'incorrect'},
 {'statement': "Dr. Hayes continues, 'In this case, the equilibrium price will fall while the equilibrium quantity is indeterminate. Consumers expecting lower prices will dampen current demand, putting downward pressure on prices. Meanwhile, reduced input costs may lead to an increase in supply. These conflicting forces make the equilibrium quantity indeterminate.'",
  'status': 'correct'},
 {'statement': "Dr. Hayes continues, 'In this case, the equilibrium price will rise while the equilibrium quantity is indeterminate. Consumers expecting lower prices would typically increase current demand, pushing prices up.'",
  'status': 'incorrect'},
 {'statement': "Dr. Hayes continues, 'In this situation, the equilibri

In [67]:
multichoice_comparison_text = "Please indicate which of the CANDIDATE STATEMENTS the SELECT STATEMENT most closely resembles. Only respond with a single number corresponding to the candidate statement selected. If the statement doesn't resemble any of the responses, respond with 0.\n"
multichoice_comparison_text += "SELECT STATEMENT: " + model_response + "\n"
for i,cs in enumerate(candidate_statements):
    multichoice_comparison_text += f"CANDIDATE STATEMENT {i+1}: " + cs['statement'] + "\n"

In [68]:
print(multichoice_comparison_text)

Please indicate which of the CANDIDATE STATEMENTS the SELECT STATEMENT most closely resembles. Only respond with a single number corresponding to the candidate statement selected. If the statement doesn't resemble any of the responses, respond with 0.
SELECT STATEMENT: 'Let's begin with consumer price expectations. If consumers expect lower prices in the future, they are likely to reduce current demand, waiting for prices to drop. This will lead to a leftward shift in the demand curve for corn, putting downward pressure on the equilibrium price and reducing the equilibrium quantity.'

Dr. Hayes shifts in his position and continues, 'On the supply side, falling input costs mean that producers can produce corn more cheaply. This typically shifts the supply curve to the right, as producers are willing to supply more corn at each price level, thus increasing the equilibrium quantity but again driving down the price.'

He concludes, 'In summary, both changes will likely result in a lower eq

In [69]:

parse_response_ml = [gpt_io.generate_user_json(multichoice_comparison_text)]
response = gpt_io.get_response(parse_response_ml)
#now, use the response to determine which item the
warnings.warn("this is untested, and we should probably run the parsing multiple times to ensure its accuracy")

getting response


In [91]:
#try to parse the response as a number using regex to extract the first numeric value

import re
response_text = response.choices[0].message.content

response_number_text = re.match(r'\d+', response_text)
if response_number_text is not None:
    response_number = int(response_number_text[0])

response_number
response_index = response_number - 1


In [92]:
if response_index>=0 and response_index<len(candidate_statements):
    parsed_choice = candidate_statements[response_index]
else:
    parsed_choice = {'statement':'', 'status':'incorrect'}

def check_item_selected(response_index:int):
#now see if response_number identifieds the correct response in the candidate statements
    #response_index = response_number-1
    if response_index>0 and response_index<=len(candidate_statements):
        if candidate_statements[response_index]['status']=='correct':
            return True
    return False

check_item_selected(response_number)


False

In order to debug and verify all this code, we'll need to gather the information together, including:

1. Original question and answers
2. Vignette (full text)
3. Raw response
4. Parsed selection
5. Model correctness

In [85]:
contextual_test_json.keys()

dict_keys(['job_title', 'vignette', 'instruction', 'dialogue_setup', 'dialogue_correct', 'dialogue_incorrect'])

In [ ]:
#try to parse the response as a number using regex to extract the first numeric value

import re
response_text = response.choices[0].message.content

response_number_text = re.match(r'\d+', response_text)
if response_number_text is not None:
    response_number = int(response_number_text[0])

response_number
response_index = response_number - 1


In [104]:
contextual_test_json

{'job_title': 'Agricultural Economist',
 'vignette': 'Dr. Hayes, an agricultural economist, has been invited to present his market forecast at an annual agriculture conference. The audience comprises farmers, researchers, and agricultural business stakeholders who are eager to understand market trends and make informed business decisions. As part of his presentation, Dr. Hayes discusses the potential impacts on market equilibrium given recent fluctuations in consumer expectations and input costs for corn production.',
 'instruction': 'You are role-playing as Dr. Hayes, an agricultural economist. Your task is to explain the effects on the equilibrium price and quantity of corn given the recent changes in consumer price expectations and falling input costs.',
 'dialogue_setup': "Dr. Hayes steps up to the podium, adjusts his glasses, and addresses the expectant audience. 'Ladies and gentlemen, given the changes in market dynamics, it's crucial to understand how these will influence the eq

In [108]:

questions = [{'q_text':test_question}]
transformation_tasks = [{'trans_task':transform_instructions}]



tranformation_test = {
    'response_raw_text':vignette_raw_response,
    'parsed_choice_text': parsed_choice['statement'],
    'choice_status': parsed_choice['status']    
}
question_transformation = {
    'transformation':contextual_test_json,
    'tranformation_tests':[tranformation_test]
}
question_map = {
    'question_orig':questions[0],
    'transformation_instructions':transformation_tasks[0],
    'question_transformations':[question_transformation]
}




question_list = [question_map]


In [112]:
question_list[0]['transformation_instructions']['trans_task']

'For each question given, follow the following steps to transform the question into an ecologically-valid scenario, outputting the result as a json dictionary:\n(1) Job Title: Identify the job title for a hypothetical employee who would have to know the answer to the question.\n(2) Vignette: Write one-paragraph vignette setting up a specific scenario where the employee uses the information--the scenario should set up the setting but not yet show usage of the information.\n(3) Instruction: Generate a system instruction for an LLM telling the LLM to role-play the employee in question.\n(4) Dialogue: Generate some character dialogue where the employee uses the information, along with some alternatives, in teh following way:\n    (4a) Generate some dialogue setup where the scenario is set up and the employee is about to use correctly use the knowledge in the correct answer, but without any text that distinguishes the correct answer from the incorrect one.\n    (4b) Then generate the dialog

In [113]:
#now flatten transformation_test into a single pandas row, handling the nested jsons
row_list = []
for qm_i, qm in enumerate(question_list):
    for qt_i, qt in enumerate(q['question_transformation']):
        for tt_i, tt in enumerate(qt['tranformation_tests']):
            #now create the pandas row, storing all the important data including the indices
            row = {}
            row['question_map_i'] = qm_i
            #should maybe be storing the questions as a dataframe to give them proper IDs but lets' not worry for now.
            #I don't want to design a whole in-memory database for this.    
            row['question_orig'] = qm['question_orig']['q_text']
            row['trans_task'] = qm['transformation_instructions']['trans_task']
            #iterate through each key in contextual_test_json and add that key to the row
            row['transformation_i'] = qt_i
            for k,v in contextual_test_json.items():
                row['transformation_'+k] = v

            row['test_i']=tt_i
            row['response_raw_text'] = tt['response_raw_text']
            row['parsed_choice_text'] = tt['parsed_choice_text']
            row['choice_status'] = tt['choice_status']
            #add the row to the list of rows
            row_list.append(row)
#convert the row_list into a pandas dataframe
df = pd.DataFrame(row_list)





In [116]:
df.to_csv("data/test_out.csv")